In [88]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score  
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Load the training data
df = pd.read_csv('/smoke_detection_iot.csv')
df

,Unnamed: 0,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,0,1654733331,20.000,57.36,0,400,12306,18520,939.735,0.00,0.00,0.00,0.000,0.000,0,0
1,1,1654733332,20.015,56.67,0,400,12345,18651,939.744,0.00,0.00,0.00,0.000,0.000,1,0
2,2,1654733333,20.029,55.96,0,400,12374,18764,939.738,0.00,0.00,0.00,0.000,0.000,2,0
3,3,1654733334,20.044,55.28,0,400,12390,18849,939.736,0.00,0.00,0.00,0.000,0.000,3,0
4,4,1654733335,20.059,54.69,0,400,12403,18921,939.744,0.00,0.00,0.00,0.000,0.000,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62625,62625,1655130047,18.438,15.79,625,400,13723,20569,936.670,0.63,0.65,4.32,0.673,0.015,5739,0
62626,62626,1655130048,18.653,15.87,612,400,13731,20588,936.678,0.61,0.63,4.18,0.652,0.015,5740,0
62627,62627,1655130049,18.867,15.84,627,400,13725,20582,936.687,0.57,0.60,3.95,0.617,0.014,5741,0
62628,62628,1655130050,19.083,16.04,638,400,13712,20566,936.680,0.57,0.59,3.92,0.611,0.014,5742,0


In [89]:
#Drop Unnamed from DataFrame
X = df.drop(['Unnamed: 0'] , axis = 1)
X

,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,1654733331,20.000,57.36,0,400,12306,18520,939.735,0.00,0.00,0.00,0.000,0.000,0,0
1,1654733332,20.015,56.67,0,400,12345,18651,939.744,0.00,0.00,0.00,0.000,0.000,1,0
2,1654733333,20.029,55.96,0,400,12374,18764,939.738,0.00,0.00,0.00,0.000,0.000,2,0
3,1654733334,20.044,55.28,0,400,12390,18849,939.736,0.00,0.00,0.00,0.000,0.000,3,0
4,1654733335,20.059,54.69,0,400,12403,18921,939.744,0.00,0.00,0.00,0.000,0.000,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62625,1655130047,18.438,15.79,625,400,13723,20569,936.670,0.63,0.65,4.32,0.673,0.015,5739,0
62626,1655130048,18.653,15.87,612,400,13731,20588,936.678,0.61,0.63,4.18,0.652,0.015,5740,0
62627,1655130049,18.867,15.84,627,400,13725,20582,936.687,0.57,0.60,3.95,0.617,0.014,5741,0
62628,1655130050,19.083,16.04,638,400,13712,20566,936.680,0.57,0.59,3.92,0.611,0.014,5742,0


In [90]:
# Choose Target
y = df['Fire Alarm']

In [91]:
# Split X, Y into Training and test sets
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.2) 

In [92]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(12, input_dim=X.shape[1], activation='relu')) 
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [93]:
# Compile the model     
model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])


from keras.callbacks import Callback
# check for accuracy repetition
class EarlyStoppingAccuracy(Callback):
    def __init__(self):
        self.counter = 0
        self.best_accuracy = 0
        
    def on_epoch_end(self, epoch, logs=None):
        accuracy = logs['accuracy']
        
        if accuracy > self.best_accuracy:
            self.best_accuracy = accuracy
            self.counter = 0
        else:       
             self.counter += 1
             
        if self.counter >= 10:
            self.model.stop_training = True

In [94]:
# Fit the model
early_stop = EarlyStoppingAccuracy() #check for accuracy repetition

model.fit(X_train, y_train, epochs=150, callbacks=[early_stop])

Epoch 1/150
1566/1566 [==============================] - 4s 2ms/step - loss: 679471.3125 - accuracy: 0.5976
Epoch 2/150
1566/1566 [==============================] - 3s 2ms/step - loss: 310487.8438 - accuracy: 0.5930
Epoch 3/150
1566/1566 [==============================] - 4s 2ms/step - loss: 248622.0312 - accuracy: 0.5935
Epoch 4/150
1566/1566 [==============================] - 3s 2ms/step - loss: 277546.0938 - accuracy: 0.5966
Epoch 5/150
1566/1566 [==============================] - 3s 2ms/step - loss: 241837.8438 - accuracy: 0.6026
Epoch 6/150
1566/1566 [==============================] - 3s 2ms/step - loss: 230045.2188 - accuracy: 0.6033
Epoch 7/150
1566/1566 [==============================] - 3s 2ms/step - loss: 194562.3750 - accuracy: 0.6071
Epoch 8/150
1566/1566 [==============================] - 3s 2ms/step - loss: 211638.9219 - accuracy: 0.6038
Epoch 9/150
1566/1566 [==============================] - 3s 2ms/step - loss: 250083.1250 - accuracy: 0.6054
Epoch 10/150
1566/1566 [====

In [95]:
# test the model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
accuracy = accuracy_score(y_test, y_pred)

392/392 [==============================] - 1s 2ms/step


In [98]:
print(f"Accuracy: {accuracy*100}")



Accuracy: 73.04007664058759
